# Hybrid Approach with Gradinet Boosting 

* Goal: build a recommender by combining user and item features
* Engineer user and product features
* Split Data with a global temporal split
* Encode product text features
* Train a gradinet boosting model (CatboostRanker) with a PairLogit loss so that it is able to rank items based on their relevance to users

In [1]:
from collections import defaultdict

import numpy as np
import pandas as pd
from catboost import CatBoostRanker, Pool
from datasets import load_dataset
from sklearn.metrics import ndcg_score
from sentence_transformers import SentenceTransformer
from tqdm import tqdm

from feature_engineering import calculate_rolling_stats
from text_pre_processing import combine_text_features, pre_process_text
from train_test_split import (
    global_temporal_split,
    temporal_split_users_in_both_sets,
    temporal_split_users_with_cold_start,
)

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
# pd.set_option('display.max_colwidth', None)

2025-01-12 22:04:40.090697: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-12 22:04:40.100084: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1736719480.110748  171897 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1736719480.114171  171897 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-12 22:04:40.125918: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

# Load Data

In [2]:
dataset_reviews = load_dataset("McAuley-Lab/Amazon-Reviews-2023", "raw_review_All_Beauty", trust_remote_code=True)
df_reviews = dataset_reviews["full"].to_pandas()

dataset_items = load_dataset("McAuley-Lab/Amazon-Reviews-2023", "raw_meta_All_Beauty", split="full", trust_remote_code=True)
df_items = dataset_items.to_pandas()

# filter out users with low ammount of reviews for now - look at the cold start problem later
min_ammount_reviews = 5
user_review_counts = df_reviews.groupby('user_id').size()
users_with_min_reviews = user_review_counts[user_review_counts >= min_ammount_reviews].index
df_reviews_filtered = df_reviews[df_reviews['user_id'].isin(users_with_min_reviews)]

df = pd.merge(df_reviews_filtered, df_items, on='parent_asin', how='left', suffixes=('_review', '_item'))

In [3]:
df.shape

(14984, 25)

In [4]:
df.columns

Index(['rating', 'title_review', 'text', 'images_review', 'asin',
       'parent_asin', 'user_id', 'timestamp', 'helpful_vote',
       'verified_purchase', 'main_category', 'title_item', 'average_rating',
       'rating_number', 'features', 'description', 'price', 'images_item',
       'videos', 'store', 'categories', 'details', 'bought_together',
       'subtitle', 'author'],
      dtype='object')

# Pre-processing

In [5]:
# item features
df['title_description_features'] = df.apply(
    lambda row: combine_text_features(row, col1='title_item', col2='features', col3='description'),
    axis=1
)
df = pre_process_text(df, input_col='title_description_features', output_col='product_title_description_feature_pre_processed')


In [6]:
df.head()

,rating,title_review,text,images_review,asin,parent_asin,user_id,timestamp,helpful_vote,verified_purchase,main_category,title_item,average_rating,rating_number,features,description,price,images_item,videos,store,categories,details,bought_together,subtitle,author,title_description_features,product_title_description_feature_pre_processed
0,5.0,Great for at home use and so easy to use!,This is perfect for my between salon visits. I...,[],B08P2DZB4X,B08P2DZB4X,AFSKPY37N3C43SOI5IEXEK5JSIYA,1627391044559,0,False,All Beauty,NIRA Skincare Laser & Serum Bundle - Includes ...,3.8,109,[POWERFUL ANTI-AGING DUO - This powerful anti-...,[],None,{'hi_res': ['https://m.media-amazon.com/images...,"{'title': [], 'url': [], 'user_id': []}",Nira,[],"{""Skin Type"": ""Dry"", ""Product Benefits"": ""Hydr...",None,None,None,NIRA Skincare Laser & Serum Bundle - Includes ...,nira skincare laser serum bundle includes anti...
1,5.0,Nice shampoo for the money,I get Keratin treatments at the salon at least...,[],B086QY6T7N,B086QY6T7N,AFSKPY37N3C43SOI5IEXEK5JSIYA,1626614511145,0,False,All Beauty,Caroline Keller Keratin Shampoo for dry and da...,3.5,12,[],[],None,{'hi_res': ['https://m.media-amazon.com/images...,"{'title': [], 'url': [], 'user_id': []}",Caroline Keller,[],"{""Brand"": ""Caroline Keller"", ""Item Form"": ""Liq...",None,None,None,Caroline Keller Keratin Shampoo for dry and da...,caroline keller keratin shampoo dry damaged ha...
2,3.0,Not what I thought I would be getting,I was very disappointed when I got this facial...,[],B08DHTJ25J,B08DHTJ25J,AFSKPY37N3C43SOI5IEXEK5JSIYA,1626211245370,0,False,All Beauty,Orange Peel Nature's Cleanse Facial Scrub - by...,3.1,12,[],[],None,{'hi_res': ['https://m.media-amazon.com/images...,"{'title': [], 'url': [], 'user_id': []}",Visage Pure,[],"{""Brand"": ""Visage Pure"", ""Skin Type"": ""Acne Pr...",None,None,None,Orange Peel Nature's Cleanse Facial Scrub - by...,orange peel nature cleanse facial scrub visage...
3,5.0,A little goes a long way!,This is a really nice moisturizing lotion. It ...,[],B07RBSLNFR,B07RBSLNFR,AFSKPY37N3C43SOI5IEXEK5JSIYA,1621184430697,0,False,All Beauty,OGANA CELL Peptide Concentrating Amazing Lotio...,4.5,26,[],[],None,{'hi_res': ['https://m.media-amazon.com/images...,"{'title': [], 'url': [], 'user_id': []}",OGANA CELL,[],"{""Brand"": ""OGANA CELL"", ""Item Form"": ""Lotion"",...",None,None,None,OGANA CELL Peptide Concentrating Amazing Lotio...,ogana cell peptide concentrating amazing lotio...
4,3.0,Just ok,I try to get Keratin treatments every 3 months...,[],B07SLFWZKN,B07SLFWZKN,AFSKPY37N3C43SOI5IEXEK5JSIYA,1619737501209,0,False,All Beauty,Keratin Secrets Do It Yourself Home Keratin Sy...,3.3,25,[],[],49.95,{'hi_res': ['https://m.media-amazon.com/images...,{'title': ['Keratin Secrets DIY Treatment Syst...,Keratin Secrets,[],"{""Package Dimensions"": ""8.27 x 4.21 x 3.9 inch...",None,None,None,Keratin Secrets Do It Yourself Home Keratin Sy...,keratin secret home keratin system keratin sec...


In [7]:
def analyze_nulls(df, column):
    total_rows = len(df)
    null_count = df[column].isnull().sum()
    null_percentage = (null_count / total_rows) * 100
    
    return {
        'column': column,
        'total_rows': total_rows,
        'null_count': null_count,
        'null_percentage': round(null_percentage, 2)
    }


In [8]:
analyze_nulls(df, 'price')

{'column': 'price',
 'total_rows': 14984,
 'null_count': 0,
 'null_percentage': 0.0}

In [9]:
# too many missing values, we won't use it for now
df[df.price == 'None'].shape[0] / df.shape[0]

0.8312867058195409

In [10]:
analyze_nulls(df, 'verified_purchase')

{'column': 'verified_purchase',
 'total_rows': 14984,
 'null_count': 0,
 'null_percentage': 0.0}

In [11]:
df['verified_purchase'] = df.verified_purchase.astype('int')

In [12]:
analyze_nulls(df, 'store')

{'column': 'store',
 'total_rows': 14984,
 'null_count': 772,
 'null_percentage': 5.15}

In [13]:
df['store'] = df['store'].fillna('UNKNOWN')

In [14]:
def pre_process_categories(row, col):
    if isinstance(row[col], str) and row[col].strip():
        categories = row[col]
    elif isinstance(row[col], (list, np.ndarray)):
        categories = ' '.join(str(element) for element in row[col])
    else:
        categories = ''
    return categories

In [15]:
df['categories_processed'] = df.apply(
    lambda row: pre_process_categories(row, col='categories'),
    axis=1
)

In [16]:
df[df.categories_processed == ''].shape

(14984, 28)

In [17]:
# maybe prefereences change over the years 
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
df['year'] = df['timestamp'].dt.year

# Feature Engineering

In [18]:
df = calculate_rolling_stats(df, 'user_id')  
df = calculate_rolling_stats(df, 'parent_asin')  
df = df.sort_values(by='timestamp')

In [19]:
df[df.user_id == 'AHV6QCNBJNSGLATP56JAWJ3C4G2A'][['timestamp', 'rating', 'rolling_avg_rating_user', 'rolling_review_count_user']].head()

,timestamp,rating,rolling_avg_rating_user,rolling_review_count_user
65,2014-11-17 18:55:42,5.0,5.000000,1
64,2015-02-17 19:11:32,5.0,5.000000,2
63,2015-04-24 21:43:07,4.0,4.666667,3
62,2015-09-12 00:04:28,5.0,4.750000,4
61,2016-02-04 22:48:30,4.0,4.600000,5


In [20]:
df[df.parent_asin == 'B0B5XFVSXY'][['timestamp', 'rating', 'rolling_avg_rating_product', 'rolling_review_count_product']].head()

,timestamp,rating,rolling_avg_rating_product,rolling_review_count_product
7768,2022-08-02 19:09:49.390,5.0,5.000000,1
3876,2022-08-12 16:44:39.817,4.0,4.500000,2
6210,2022-08-17 12:17:13.185,2.0,3.666667,3
14166,2022-08-20 17:40:53.209,5.0,4.000000,4
10252,2022-08-25 21:04:21.033,5.0,4.200000,5


# Select Features

In [21]:
# numerical features
user_review_features = ['rolling_avg_rating_user', 'rolling_review_count_user', 'helpful_vote', 'verified_purchase', 'year']
product_features = ['average_rating', 'rolling_avg_rating_product', 'rolling_review_count_product', 'rating_number']
categoricals = ['main_category', 'store']
text_features = ['product_title_description_feature_pre_processed']
target = 'rating'

train_test_split_features =  ['timestamp', 'parent_asin']
group_features = ['user_id']

In [22]:
columns = train_test_split_features + group_features + user_review_features + product_features + categoricals + text_features + [target]

In [23]:
df = df[columns]

In [24]:
df.head()

,timestamp,parent_asin,user_id,rolling_avg_rating_user,rolling_review_count_user,helpful_vote,verified_purchase,year,average_rating,rolling_avg_rating_product,rolling_review_count_product,rating_number,main_category,store,product_title_description_feature_pre_processed,rating
778,2004-08-15 20:42:27,B00014DMLO,AGWDYYVVWM3DC3CASUZKXK67G6IA,5.0,1,20,0,2004,4.0,5.0,1,15,All Beauty,Burt's Bees,burt bee marshmallow cream oz burt bee marshma...,5.0
1498,2004-08-23 17:22:10,B0001ZI0BU,AFIJLAW3HIOMRUFSWNH54IJ3XQAA,4.0,1,30,0,2004,3.0,4.0,1,29,All Beauty,Crest,crest whitestrips premium dental whitening sys...,4.0
777,2004-12-11 08:04:29,B00064A816,AGWDYYVVWM3DC3CASUZKXK67G6IA,5.0,2,1,0,2004,5.0,5.0,1,7,All Beauty,Zia,zia natural skincare ultimate body butter oz z...,5.0
776,2005-06-28 16:51:01,B00094DTSW,AGWDYYVVWM3DC3CASUZKXK67G6IA,5.0,3,2,0,2005,4.0,5.0,1,15,All Beauty,Burt's Bees,burt bee marshmallow cream oz burt bee marshma...,5.0
460,2005-08-09 12:12:58,B000068PBJ,AGZZXSMMS4WRHHJRBUJZI4FZDHKQ,5.0,1,5,1,2005,4.0,5.0,1,154,All Beauty,Norelco,norelco xl norelco xl hightech cordcordless sh...,5.0


# Train Test Split

In [25]:
train_df, test_df = global_temporal_split(df, split_ratio=0.8, exclude_cold_start_users=False)

In [26]:
train_df.shape

(11987, 16)

In [27]:
test_df.shape

(2997, 16)

In [28]:
train_df.user_id.unique().shape

(1479,)

In [29]:
test_df.user_id.unique().shape

(749,)

In [30]:
# 141 users only have interactions in the test set
cold_start_users = [i for i in test_df.user_id.unique() if i not in train_df.user_id.unique()]
len(cold_start_users)

141

In [31]:
# 83% of products in the test set not in the train set
# we will try to work wit this by adding content-based features
test_only_products = [i for i in test_df.parent_asin.unique() if i not in train_df.parent_asin.unique()]
len(test_only_products) / len(test_df.parent_asin.unique())

0.8361153262518968

In [32]:
len(train_df.parent_asin.unique())

5880

In [33]:
train_df = train_df.drop(train_test_split_features, axis=1)
test_df = test_df.drop(train_test_split_features, axis=1)

# Generate Embeddings For Text Features

In [34]:
model = SentenceTransformer('all-MiniLM-L6-v2')

In [35]:
train_embeddings = model.encode(train_df['product_title_description_feature_pre_processed'].tolist(), batch_size=64, show_progress_bar=True)
test_embeddings = model.encode(test_df['product_title_description_feature_pre_processed'].tolist(), batch_size=64, show_progress_bar=True)

Batches:   0%|          | 0/188 [00:00<?, ?it/s]

Batches:   0%|          | 0/47 [00:00<?, ?it/s]

In [36]:
np.save('train_embeddings.npy', train_embeddings)
np.save('test_embeddings.npy', test_embeddings)

In [37]:
train_embeddings = np.load('train_embeddings.npy')
test_embeddings = np.load('test_embeddings.npy')

In [38]:
embeding_columns = [f'product_embeddings_{i}' for i in range(train_embeddings.shape[1])]

In [39]:
train_embeddings_df = pd.DataFrame(train_embeddings, columns=embeding_columns)
test_embeddings_df = pd.DataFrame(test_embeddings, columns=embeding_columns)

In [40]:
train_df = pd.concat([train_df, train_embeddings_df], axis=1)
test_df = pd.concat([test_df, test_embeddings_df], axis=1)

In [41]:
train_df.head()

,user_id,rolling_avg_rating_user,rolling_review_count_user,helpful_vote,verified_purchase,year,average_rating,rolling_avg_rating_product,rolling_review_count_product,rating_number,main_category,store,product_title_description_feature_pre_processed,rating,product_embeddings_0,product_embeddings_1,product_embeddings_2,product_embeddings_3,product_embeddings_4,product_embeddings_5,product_embeddings_6,product_embeddings_7,product_embeddings_8,product_embeddings_9,product_embeddings_10,product_embeddings_11,product_embeddings_12,product_embeddings_13,product_embeddings_14,product_embeddings_15,product_embeddings_16,product_embeddings_17,product_embeddings_18,product_embeddings_19,product_embeddings_20,product_embeddings_21,product_embeddings_22,product_embeddings_23,product_embeddings_24,product_embeddings_25,product_embeddings_26,product_embeddings_27,product_embeddings_28,product_embeddings_29,product_embeddings_30,product_embeddings_31,product_embeddings_32,product_embeddings_33,product_embeddings_34,product_embeddings_35,product_embeddings_36,product_embeddings_37,product_embeddings_38,product_embeddings_39,product_embeddings_40,product_embeddings_41,product_embeddings_42,product_embeddings_43,product_embeddings_44,product_embeddings_45,product_embeddings_46,product_embeddings_47,product_embeddings_48,product_embeddings_49,product_embeddings_50,product_embeddings_51,product_embeddings_52,product_embeddings_53,product_embeddings_54,product_embeddings_55,product_embeddings_56,product_embeddings_57,product_embeddings_58,product_embeddings_59,product_embeddings_60,product_embeddings_61,product_embeddings_62,product_embeddings_63,product_embeddings_64,product_embeddings_65,product_embeddings_66,product_embeddings_67,product_embeddings_68,product_embeddings_69,product_embeddings_70,product_embeddings_71,product_embeddings_72,product_embeddings_73,product_embeddings_74,product_embeddings_75,product_embeddings_76,product_embeddings_77,product_embeddings_78,product_embeddings_79,product_embeddings_80,product_embeddings_81,product_embeddings_82,product_embeddings_83,product_embeddings_84,product_embeddings_85,product_embeddings_86,product_embeddings_87,product_embeddings_88,product_embeddings_89,product_embeddings_90,product_embeddings_91,product_embeddings_92,product_embeddings_93,product_embeddings_94,product_embeddings_95,product_embeddings_96,product_embeddings_97,product_embeddings_98,product_embeddings_99,product_embeddings_100,product_embeddings_101,product_embeddings_102,product_embeddings_103,product_embeddings_104,product_embeddings_105,product_embeddings_106,product_embeddings_107,product_embeddings_108,product_embeddings_109,product_embeddings_110,product_embeddings_111,product_embeddings_112,product_embeddings_113,product_embeddings_114,product_embeddings_115,product_embeddings_116,product_embeddings_117,product_embeddings_118,product_embeddings_119,product_embeddings_120,product_embeddings_121,product_embeddings_122,product_embeddings_123,product_embeddings_124,product_embeddings_125,product_embeddings_126,product_embeddings_127,product_embeddings_128,product_embeddings_129,product_embeddings_130,product_embeddings_131,product_embeddings_132,product_embeddings_133,product_embeddings_134,product_embeddings_135,product_embeddings_136,product_embeddings_137,product_embeddings_138,product_embeddings_139,product_embeddings_140,product_embeddings_141,product_embeddings_142,product_embeddings_143,product_embeddings_144,product_embeddings_145,product_embeddings_146,product_embeddings_147,product_embeddings_148,product_embeddings_149,product_embeddings_150,product_embeddings_151,product_embeddings_152,product_embeddings_153,product_embeddings_154,product_embeddings_155,product_embeddings_156,product_embeddings_157,product_embeddings_158,product_embeddings_159,product_embeddings_160,product_embeddings_161,product_embeddings_162,product_embeddings_163,product_embeddings_164,product_embeddings_165,product_embeddings_166,product_embedd

In [42]:
train_df = train_df.drop(text_features, axis=1)
test_df = test_df.drop(text_features, axis=1)

# Train Model

In [43]:
features = user_review_features + product_features + categoricals + embeding_columns

In [44]:
y_train = train_df[target]
X_train = train_df.drop([target], axis=1)

y_test = test_df[target]
X_test = test_df.drop([target], axis=1)

In [49]:
categorical_indices = [X_train[features].columns.get_loc(col) for col in categoricals]
categorical_indices

[9, 10]

In [50]:
train_df = train_df.sort_values(by='user_id')
test_df = test_df.sort_values(by='user_id')

In [51]:
train_pool = Pool(
    data=train_df[features],
    label=train_df[target],
    group_id=train_df['user_id'].tolist(), 
    cat_features=categorical_indices
)

test_pool = Pool(
    data=test_df[features],
    label=test_df[target],
    group_id=test_df['user_id'].tolist(),
    cat_features=categorical_indices 
)


In [52]:
model = CatBoostRanker(
    loss_function='PairLogit',
    iterations=1000,
    #learning_rate=0.1,
    #depth=6,
    cat_features=categorical_indices
)

model.fit(train_pool,
        eval_set=test_pool,
        metric_period=30,
        use_best_model=True 
          
         )

Groupwise loss function. OneHotMaxSize set to 10
0:	learn: 0.6668149	test: 0.6684777	best: 0.6684777 (0)	total: 111ms	remaining: 1m 51s
30:	learn: 0.2929121	test: 0.3139320	best: 0.3139320 (30)	total: 1.71s	remaining: 53.6s
60:	learn: 0.2404357	test: 0.2676233	best: 0.2676233 (60)	total: 3.3s	remaining: 50.9s
90:	learn: 0.2165997	test: 0.2484674	best: 0.2484674 (90)	total: 4.93s	remaining: 49.3s
120:	learn: 0.2045301	test: 0.2432725	best: 0.2432725 (120)	total: 6.55s	remaining: 47.6s
150:	learn: 0.1925546	test: 0.2336399	best: 0.2336399 (150)	total: 8.22s	remaining: 46.2s
180:	learn: 0.1838758	test: 0.2286192	best: 0.2286192 (180)	total: 9.84s	remaining: 44.5s
210:	learn: 0.1768618	test: 0.2265012	best: 0.2265012 (210)	total: 11.4s	remaining: 42.6s
240:	learn: 0.1713312	test: 0.2248301	best: 0.2248301 (240)	total: 13s	remaining: 40.8s
270:	learn: 0.1664619	test: 0.2237424	best: 0.2237424 (270)	total: 14.5s	remaining: 39s
300:	learn: 0.1584116	test: 0.2185686	best: 0.2185686 (300)	total

In [53]:
feature_importances = model.get_feature_importance(train_pool)
for score, name in sorted(zip(feature_importances, features), reverse=True):
    print('{}: {}'.format(name, score))

rolling_avg_rating_product: 0.34717154502987063
rolling_avg_rating_user: 0.023617360333630072
rolling_review_count_product: 0.007801716286095425
rolling_review_count_user: 0.004556889765504224
product_embeddings_2: 0.0004390898369351359
product_embeddings_41: 0.0004277715162201434
helpful_vote: 0.00041590934700291154
product_embeddings_265: 0.0003968256721671587
product_embeddings_13: 0.0003601350723804464
product_embeddings_211: 0.0003526371483202895
product_embeddings_322: 0.00034305957363919143
product_embeddings_182: 0.00033366277523071836
product_embeddings_254: 0.0003167962340359376
product_embeddings_6: 0.000308839003129259
product_embeddings_325: 0.00030693182414166104
product_embeddings_109: 0.000303290930065786
product_embeddings_38: 0.00029821406633600417
product_embeddings_190: 0.0002963807297866744
product_embeddings_331: 0.00029384633144120126
product_embeddings_314: 0.000290796135021823
product_embeddings_199: 0.0002845296114136406
product_embeddings_244: 0.0002747570134

# Evaluate

In [54]:
test_df["predicted_score"] = model.predict(test_pool)

In [55]:
ndcg_scores = []
num_users = 0
for user_id in test_df['user_id'].unique():
    true_relevance = test_df[test_df['user_id'] == user_id][target].tolist()
    predicted_scores = test_df[test_df['user_id'] == user_id]['predicted_score'].tolist()
    if len(predicted_scores) > 1:
        user_ndcg = ndcg_score([true_relevance], [predicted_scores], k=10)
        ndcg_scores.append(user_ndcg)
        num_users += 1

average_ndcg = np.mean(ndcg_scores)
print(f"Average NDCG@10 across all users with mora than 1 rating ({num_users}, {num_users/test_df.user_id.unique().shape[0]}% of the test set): {average_ndcg:.4f}")


Average NDCG@10 across all users with mora than 1 rating (521, 0.6955941255006676% of the test set): 0.9961


In [56]:
def precision_recall_at_k(group, k=10):
    # sort by predicted_score descending
    group_sorted = group.sort_values("predicted_score", ascending=False)
    
    top_k = group_sorted.head(k)
    
    # number of relevant items in the top K
    relevant_in_top_k = top_k["relevant"].sum()
    
    # total relevant items for this user
    total_relevant = group["relevant"].sum()
    
    precision_k = relevant_in_top_k / k
    recall_k = relevant_in_top_k / total_relevant if total_relevant > 0 else 0.0
    
    return pd.Series({
        "precision@{}".format(k): precision_k,
        "recall@{}".format(k): recall_k
    })

def compute_precision_recall_at_k(df, user_col="user_id", k=10):
    """ 
    Compute mean precision@K and recall@K across all users. 
    """
    metrics_df = (
        df
        .groupby(user_col)
        [df.columns]
        .apply(lambda g: precision_recall_at_k(g, k))
    )

    return metrics_df.mean().to_dict()



In [57]:
test_df["relevant"] = (test_df["rating_number"] >= 4).astype(int)

test_df_sorted = (
    test_df
    .groupby("user_id", group_keys=False)
    [['user_id', 'relevant', 'predicted_score']] 
    .apply(lambda df: df.sort_values("predicted_score", ascending=False))
)

metrics_k10 = compute_precision_recall_at_k(test_df_sorted, user_col="user_id", k=10)
print(metrics_k10)

{'precision@10': 0.35193591455273626, 'recall@10': 0.950020205227441}
